In [28]:
from __future__ import print_function
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib as plt
import csv
import numpy as np
import pandas as pd
import textmining
import lda
import textmining
import nltk.corpus
import re
import seaborn as sns

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
with open("NSF_test.csv") as f:
    Header = f.next().split(',')
number_name = Header[0]
text_name = Header[1][:-2]

In [3]:
input_file = csv.DictReader(open("NSF_test.csv"))

In [4]:
text = []
number = []
for row in input_file:
    number.append(row[number_name])
    text.append(row[text_name])

In [5]:
text_stop = []
stop = set(stopwords.words('english'))

text_stop = []
stop = set(stopwords.words('english'))
stop |= {'br', 'research', 'new', 'made', 'use'}

for i in range(len(text)):
    sentence = text[i].lower()
    A = [i for i in sentence.replace('<br/>',' ').split() if i not in stop]
    A = [i[:-1] for i in A  if i[-1] == 's']
    #A = [wordnet.lemmatize(word) for word in sentence]
    text_stop.append(' '.join(A))

In [6]:
A

['inne',
 'dougla',
 'solti',
 'solti',
 'flavor',
 'member',
 'by-product',
 'flavor',
 'tea',
 'compound',
 'support',
 'compound',
 'studie',
 'pathway',
 'compound',
 'factor',
 'dataset',
 'acros',
 'processe',
 'require',
 'expert',
 'genetic',
 'student',
 'frontier',
 'genomic',
 'link',
 'product',
 "children'",
 'offer',
 'display',
 'tour',
 'scientist',
 'represent',
 'angiosperm',
 'subfamilie',
 'difference',
 'synthesi',
 'iridoid',
 'compound',
 'flavor',
 'integrate',
 'dataset',
 'pathway',
 'transcriptome',
 'metabolome',
 'specie',
 'specie',
 'level',
 'thu',
 'resource',
 'gene',
 'event',
 'mechanism',
 'hypothese',
 'gene',
 'resource',
 'sequence']

In [7]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

docs = text_stop[0].split()

docs_porter = [porter.stem(word) for word in docs]
               
docs_snowball = [snowball.stem(word) for word in docs]

docs_wordnet = [wordnet.lemmatize(word) for word in docs]


In [8]:
docs_wordnet

['infectiou',
 'dendrobatidi',
 'spread',
 'salamander',
 '(genu',
 'show',
 'population',
 'investigator',
 'hypothesi',
 'increase',
 'level',
 'researcher',
 'hypothesi',
 'historie',
 'los',
 "population'",
 'infectiou',
 'implication',
 'insight',
 'undergraduate',
 'minoritie',
 'group',
 'student',
 'tool',
 'skill',
 'challenge',
 'student',
 'receive',
 'querie',
 'finding',
 'publication',
 'presentation',
 'pi']

In [9]:
d = len(text)
n_topics = 30

In [10]:
	for i in range(len(text)):
	    sentence = text[i].lower()
	    A = [i for i in sentence.split() if i not in stop]
	    text_stop.append(' '.join(A))

	tdm = textmining.TermDocumentMatrix()

	for i in range(d):
	    tdm.add_doc(text_stop[i])

	# # create a temp variable with doc-term info
	temp = list(tdm.rows(cutoff=1))

	# get the vocab from first row
	vocab = tuple(temp[0])

	# # get document-term matrix from remaining rows
	X = np.array(temp[1:])


	#tdm.write_csv('matrix.csv', cutoff=1)

	Topics = []

	model = lda.LDA(n_topics, n_iter=100, random_state=1)
	model.fit(X)  # model.fit_transform(X) is also available
	topic_word = model.topic_word_  # model.components_ also works
	n_top_words = 8
	for i, topic_dist in enumerate(topic_word):
	    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
	    Topics.append('Topic {}: {}'.format(i+1, ' '.join(topic_words)))
	    print('Topic {}: {}'.format(i+1, ' '.join(topic_words)))

Topic 1: studie tool activitie processe mutation technique protein residue
Topic 2: population specie investigator gene factor insight difference individual
Topic 3: specie gene trait proces rate studie teacher organism
Topic 4: student gene mechanism activitie opportunitie variou group represent
Topic 5: student provide science project technique program event participant
Topic 6: tool goal impact resource analyse les require change
Topic 7: undergraduate effect student region heterosi fitnes combination mutation
Topic 8: microbe pattern communitie acros dynamic mean prediction goal
Topic 9: protein studie group proces channel question major use
Topic 10: method physic interaction biophysic system processe model propertie
Topic 11: mechanism genetic insect thu seed function strategie sequence
Topic 12: specie researcher scientist tree addres numerou forest u
Topic 13: relationship animal group use stream lake rate source
Topic 14: cell student molecule processe analysi propertie pathwa

In [11]:
	topic_word = model.topic_word_ 
	doc_topic = model.doc_topic_

In [12]:
	with open('topic_table.csv', 'w') as f:
	    # create header
	    #header = 'document'
	    #for k in range(n_topics):
	    #   header += ', pr_topic_{}'.format(k)
	    #f.write(header + '\n')

	    # write one row for each document
	    # col 1 : document number
	    # cols 2 -- : topic probabilities
	    for k in range(d):
	        # format probabilities into string
	        str_probs = ','.join(['{:.5e}'.format(pr) for pr in doc_topic[k,:]])
	        # write line to file
	        #f.write('{}, {}\n'.format(k, str_probs))
	        f.write('{}\n'.format(str_probs))

In [13]:
	dat = np.genfromtxt ('topic_table.csv', delimiter=",")

In [14]:
M = []
for i in range(len(number)):
    M.append(dat[i,:]*int(number[i]))
M = sum(M)

In [15]:
M

array([ 7183232.2122107 ,  7155194.8301095 ,  9588536.30739408,
        8384503.53635426,  8471701.63185506,  6806928.63669264,
        5409524.25259089,  5752135.85771036,  6628760.92671108,
        9744640.60595699,  7086477.39102612,  8697243.58989591,
        4168758.19068995,  9717119.32882624,  4635155.19235699,
        7764629.07932004,  6783969.41648073,  4664172.93170616,
        8756461.12449673,  6724847.10156213,  5598046.81024881,
        5741881.94771134,  6201336.6472921 ,  6337254.42520244,
        7939606.50318772,  7401858.76042558,  6645849.47407917,
        6475091.86599542,  7554559.22088301,  8171682.34982267])

In [21]:
x=np.arange(1, n_topics+1, 1)
y=M

In [29]:
	ax = sns.barplot(x, y, color='Blue')
	ax.set(xlabel='Topics', ylabel=number_name)
	ax.set_title('Total')
	plt.show()

AttributeError: 'module' object has no attribute 'show'